# Домашнее задание 1 [10 баллов] 
# До 1.12.17 23:59

Задание выполняется в группе (1-4 человека). В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.

Не все части обязательны для выполнения, однако вы можете быть дополнительно оштрафованы за небрежное за выполнение одной или двух частей вместо четырех.

При возниконовении проблем с выполнением задания обращайтесь с вопросами к преподавателю. Поэтому настоятельно рекомендуется выполнять задание заранее, оставив запас времени на всевозможные технические проблемы. Если вы начали читать условие в последний вечер и не успели из-за проблем с установкой какой-либо библиотеки — это ваши проблемы.


Результат выполнения задания — это отчёт в формате html на основе Jupyter Notebook. Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание **минимума** необходимой теории и/или описание используемых инструментов - не стоит переписывать лекции или Википедию
* Подробный пошаговый рассказ о проделанной работе
* Аккуратно оформленные результаты
* **Внятные выводы** – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Небрежное его оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в тексте в явном виде.

Сдача отчетов осуществляется через систему AnyTask.



## Кластеризация новостей
Входная коллекция данных состоит из двух частей:
1. events.csv – список 28 резонансных событий первой половины 2017 года, каждому событию присвоен свой порядковый номер (id)
2. raw_news.csv – тексты новостей из различных новостных источников, известно, к какому резонансному событию относится каждая новость (столбец event_id).

Будем считать, что одно событие – это один кластер. В этом домашнем задании вам предстоит:
1. провести кластеризацию текстов новостей и проверить, получается ли восстановить кластерную структуру 
2. проверить, можно ли использовать кластерный анализ для обобщения: найти небольшое число кластеров и проверить, получается ли выделить общие направлени новостей

### Часть 1 [2 балла] Предварительная обработка текстов
Проведите предобработку новостей: токенизацию, приведение к нижнему регистру, лемматизацию. Проверьте, есть ли в коллекции дубликаты. Посчитайте, сколько новостей относится к каждому резонансному событию. 

Импорт необходимых библиотек:

In [1]:
# -*- coding: utf-8 -*-
from os import path
import glob
import pickle
import gensim
import numpy as np
import pickle
import time
from nltk.tokenize import RegexpTokenizer
from re import sub
from pandas import DataFrame
from pymystem3 import Mystem
import texterra
from pymorphy2 import MorphAnalyzer

c:\program files\python35\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


ImportError: No module named 'nltk'

Было решено опробовать каждый из известных нам лемматизаторов (Mystem, pymorphy2 и Texterra) и понять, какой из них лучше работает для нашей задачи. 

Создаем экземпляры классов для каждого лемматизатора:

In [2]:
API_KEY = '9988cfb979b80264baeba1386cc7e455f99f943c'

morph = MorphAnalyzer()
m = Mystem()
t = texterra.API(API_KEY)
alpha_tokenizer = RegexpTokenizer('\w+')


NameError: name 'MorphAnalyzer' is not defined

In [ ]:
df_events = DataFrame.from_csv('events.csv')
df_news = DataFrame.from_csv('raw_news.csv')
texts = list(df_news.text.values)

In [ ]:
all_type_of_tokens = {}
tokenizers = {'pymorphy': normalize_with_pymorphy, 'mystem': normalize_with_mystem, 'texterra': normalize_with_texterra}

for name, tokenizer in tokenizers.items():
    all_texts = []
    for text in texts:
        print(texts.index(text))
        text = remove_url(text)
        tokens = alpha_tokenizer.tokenize(text)
        tokens = tokenizer(tokens)
        all_texts.append(tokens)
    with open('tokens_from_' + name + '.pickle', 'wb') as f:
        pickle.dump(all_texts, f)

In [18]:
num_of_features = 200


In [19]:
data = {}
for filename in glob.iglob('lemmas_from_*.pickle', recursive=True):
    with open(filename, 'rb') as f:
        data_lemmas = pickle.load(f)
        data[path.splitext(path.basename(filename))[0]] = data_lemmas


In [20]:
models = {}
for filename in glob.iglob('w2v_models_from_*.mdl', recursive=True):
    model = gensim.models.Word2Vec.load_word2vec_format(filename)
    models[path.splitext(path.basename(filename))[0]] = model

Что-то...

In [21]:
for name_of_alg, list_of_docs in data.items():    
vectors_list = []
    
    
for text_id in range(len(list_of_docs)):
        
    vector_for_each_text = []
    for word in list_of_docs[text_id]:
        try:
            featureVec = np.zeros(shape=(1, num_of_features), dtype='float32')
            featureVec = np.add(featureVec, models[name_of_alg][word])
            vector_for_each_text.append(featureVec)
            print(featureVec)
        except KeyError:
            pass

        #first = np.concatenate(np.matrix(vector_for_each_text), axis=0)
        #resultVec = np.divide(first, len(vector_for_each_text))
        #vectors_list.append(resultVec)

    #vectors_array = np.concatenate(np.matrix(vectors_list), axis=0)
    
    #with open(alg_name.replace('lemmas','vectors')+'.pickle', 'wb') as f:
            #pickle.dump(np.matrix(vectors_array), f)

In [11]:
np.matrix(np.array(vectors_list)).shape

ValueError: cannot reshape array of size 0 into shape (1,1930)

In [1]:
# -*- coding: utf-8 -*-
from __future__ import print_function

import codecs
from os import path
import glob
import pickle
import csv
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.metrics import *
from sklearn.cluster import *
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import *

In [5]:
d = {}
for filename in glob.iglob('vectors_from_*.pickle', recursive=True):
    with open(filename, 'rb') as f:
        data_model = pickle.load(f)
        d[path.splitext(path.basename(filename))[0].replace('vectors_from_','')] = data_model
print(d['mystem'].shape)

(903123, 200)


In [ ]:
pipeline = Pipeline([
        ('tfidf', TfidfTransformer()),
        ('svd', TruncatedSVD(n_components=150)),
        ('norm', Normalizer()),
        ('clust', AgglomerativeClustering(n_clusters=28))
    ])
pipeline.fit(data_model)

explained_variance = pipeline.named_steps['svd'].explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))


In [ ]:
clust_labels = pipeline.named_steps['clust'].labels_

    with codecs.open(raw_news_filepath, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=',', quoting=csv.QUOTE_NONE)
        labels = [row[1] for row in reader]
        labels = [int(l) for l in labels[1:]]

    print("Homogeneity:", homogeneity_score(labels, clust_labels))
    print("Completeness:", completeness_score(labels, clust_labels))
    print("V-measure",  v_measure_score(labels, clust_labels))
    print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))